# Hyperparameter Tuning using HyperDrive

In this notebook we are going to train a model using and adjust its hyperparameters using Hyperdrive.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration
from train import clean_data
import pandas as pd

## Dataset

The dataset we will be using is the "Heart failure prediction dataset" from Kaggle (https://www.kaggle.com/fedesoriano/heart-failure-prediction). This dataset tries to help in the early detection of severe heart diseases by studying the way several health indicators affect the occurrence of such diseases. This dataset is a combination of 5 different datasets about this kind of diseases (more information in the Kaggle url provided earlier). 

In [2]:
ws = Workspace.from_config()
experiment_name = 'udacityproj3hyper'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')


Workspace name: quick-starts-ws-165696
Azure region: southcentralus
Subscription id: 510b94ba-e453-4417-988b-fbdc37b55ca7
Resource group: aml-quickstarts-165696


In [3]:
cluster_name="udacityprojclust"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('cpu cluster already exist. Using it.')
except ComputeTargetException:

    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

cpu cluster already exist. Using it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

The model we will be using it is a simple Logistic Regression, this has the advantage of being quite interpretable. We will be using also hyperdrive to configure the best hyperparameters from this kind of model.
For the optimization of hyperparameters we will use random parameter sampling that provides a quick and robust way of searching the appropiate parameters, we will also implement an early stopping policy for optimizing the use of computational resources, the early stopping policy is the BanditPolicy that gets a right trade-off between saving compute resources and losing some probable good cases that would have improved the primary metric.

In [5]:
# Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.1, delay_evaluation=5)

# Create the different params that you will be using during training
param_sampling = RandomParameterSampling({'C': uniform(0.5,1.5), 'max_iter': choice(50, 75, 100, 125, 150)})

# Create your estimator and hyperdrive config
estimator = SKLearn(source_directory='./', entry_script='train.py', compute_target=cpu_cluster, script_params={'--C': 'C', '--max_iter':'max_iter'})

hyperdrive_run_config = HyperDriveConfig(hyperparameter_sampling=param_sampling, policy=early_termination_policy, primary_metric_name='AUC', max_duration_minutes=100, max_total_runs=10, primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, estimator=estimator)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [6]:
# Submit your experiment

hyperdrive_run = experiment.submit(hyperdrive_run_config)

The same input parameter(s) are specified in estimator/run_config script params and HyperDrive parameter space. HyperDrive parameter space definition will override these duplicate entries. ['--C', '--max_iter'] is the list of overridden parameter(s).


## Run Details

In the next cell we are getting the experiment run detail through the 'RunDetails' widget.

In [7]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"



## Best Model

In this next cells we are going to get the best model measured by the primary metric chosen previously. We will get also the best hyperparameters from the hyperdrive run and finally we will save it to the Azure storage.

In [8]:
# Get the data and preprocess it.

ds = pd.read_csv('./heart.csv')

x, y = clean_data(ds)

In [9]:
# Get the best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()

best_param = best_run.get_details()['runDefinition']['arguments']

print('Best parameters')
print(best_param)

LR_model = LogisticRegression(C=float(best_param[1]), max_iter=int(best_param[3]))

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

LR_model.fit(x_train, y_train)

Best parameters
['--C', '0.8449002363258162', '--max_iter', '150']


LogisticRegression(C=0.8449002363258162, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=150, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [10]:
# Save the best model

joblib.dump(LR_model, 'hyperdrive_LR.model')

['hyperdrive_LR.model']

## Model Deployment

We register the model in our Azure workspace in order to get the chance to deploy it if we need it.

In [11]:
# Register de model

model = Model.register(workspace=ws,
                       model_name='my-udacityproj3-hyperdrivemodel', # Name of the registered model in your workspace.
                       model_path='./hyperdrive_LR.model',  # Local file to upload and register as a model.
                       model_framework=Model.Framework.SCIKITLEARN,  # Framework used to create the model.
                       resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=1.0),
                       description='Hyperdrive best model for heart disease prediction.',
                       tags={'area': 'heartdisease', 'type': 'classification'})

print('Name:', model.name)
print('Version:', model.version)

Registering model my-udacityproj3-hyperdrivemodel
Name: my-udacityproj3-hyperdrivemodel
Version: 1


We deploy the model using a Azure curated environment, so there is no need for further configuration.

In [12]:
# Deploy model in a default environment as we have a scikit-learn model.

service_name = 'my-udacityproj3-hyperserv'

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       overwrite=True)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-12-13 16:38:11+00:00 Creating Container Registry if not exists.
2021-12-13 16:38:11+00:00 Registering the environment.
2021-12-13 16:38:12+00:00 Uploading autogenerated assets for no-code-deployment.
2021-12-13 16:38:13+00:00 Building image..
2021-12-13 16:43:26+00:00 Generating deployment configuration.
2021-12-13 16:43:27+00:00 Submitting deployment to compute..
2021-12-13 16:43:31+00:00 Checking the status of deployment my-udacityproj3-hyperserv..
2021-12-13 16:45:48+00:00 Checking the status of inference endpoint my-udacityproj3-hyperserv.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [13]:
print(service.get_logs())

2021-12-13T18:42:47,198400277+00:00 - rsyslog/run 
2021-12-13T18:42:47,399931593+00:00 - iot-server/run 
2021-12-13T18:42:47,397082146+00:00 - gunicorn/run 
2021-12-13T18:42:47,399933293+00:00 - nginx/run 

PATH environment variable: /azureml-envs/azureml_00066918e50f5ae481b726dc526556ed/bin:/opt/miniconda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin
PYTHONPATH environment variable: 

Pip Dependencies
---------------
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-12-13T18:42:49,903993983+00:00 - iot-server/finish 1 0
2021-12-13T18:42:50,096464345+00:00 - Exit code 1 is normal. Not restarting iot-server.
adal==1.2.7
applicationinsights==0.11.10
azure-common==1.1.27
azure-core==1.21.1
azure-graphrbac==0.61.1
azure-mgmt-authorization==2.0.0
azure-mgmt-containerregistry==8.2.0
azure-mgmt-core==1.3.0
azure-mgmt-keyvault==9.3.0
azure-mgmt-resource==20.0.0
azure-mgmt-storage==19.0.0
azureml-core==1.0.57.1
azureml-defaults==1.0.57
azureml-mo

In the next cells we are getting preprocessed data from the test set to check if the deployed model can provide predictions.

In [15]:
data_tosend = x_test[0:2].values.tolist()

print(data_tosend)

data_tosend = [data_tosend[0][:], data_tosend[1][:]]

print(data_tosend)

[[0.5510204081632653, 1.0, 0.575, 0.0, 1.0, 0.6690140845070423, 0.0, 0.3068181818181818, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0], [0.7755102040816326, 1.0, 0.6, 0.5008291873963516, 0.0, 0.6408450704225352, 0.0, 0.3409090909090909, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]]
[[0.5510204081632653, 1.0, 0.575, 0.0, 1.0, 0.6690140845070423, 0.0, 0.3068181818181818, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0], [0.7755102040816326, 1.0, 0.6, 0.5008291873963516, 0.0, 0.6408450704225352, 0.0, 0.3409090909090909, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]]


We get some predictions from the test data. The predictions are in the form of probabilities of belonging to a certain class.

In [17]:
import json

input_payload = json.dumps({
    'data': data_tosend,
    })

output = service.run(input_payload)

print(output)

{'predict_proba': [[0.17958440072181836, 0.8204155992781816], [0.42571456708992106, 0.5742854329100789]]}


We finally deallocate all the compute resources used for training and serving of this model.

In [ ]:
# Remove WebService endpoint

service.delete()

# Remove compute cluster

cpu_cluster.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

